## Implementación para 1 empresa y 1 vendedor

Suposiciones:
- Se puede producir y vender de forma indefinida.
- La empresa sube la mercancia al camión que es de tamaño finito.
- En caso de llenarse el camión la empresa espera hasta que haya espacio.
- El comprador compra desde el camión, si no hay mercancia espera.

In [33]:
import multiprocessing as mp 
import time
N = 100
camion = []
en_camion = mp.Semaphore(0)
espacios_vacios = mp.Semaphore(N)

def producir_caja():
  print("Una caja de naranjas producida!")
  return 1

def empresa_narajas(en_camion, espacios_vacios):
  for i in range(3):
    caja_naranjas = producir_caja()
    espacios_vacios.acquire()
    camion.append(caja_naranjas)
    en_camion.release()

def vender_naranjas(caja_naranjas):
    print("Una caja de naranjas vendida!")

def comprador_central(en_camion, espacios_vacios):
  for i in range(3):
    en_camion.acquire()
    caja_naranjas = camion.pop(0)
    espacios_vacios.release()
    vender_naranjas(caja_naranjas)

empresa_narajas_proceso = mp.Process(target=empresa_narajas(en_camion, espacios_vacios))
comprador_central_proceso = mp.Process(target=comprador_central(en_camion, espacios_vacios))

empresa_narajas_proceso.start()
comprador_central_proceso.start()                            

Una caja de naranjas producida!
Una caja de naranjas producida!
Una caja de naranjas producida!
Una caja de naranjas vendida!
Una caja de naranjas vendida!
Una caja de naranjas vendida!


## Implementación $n\_empresas$ y $n\_compradores$

Suposiciones:
- Se tiene $n\_emmpresas$ y $n\_compradores$.
- Las empresas comparten un camión de tamaño finito a donde suben la mercancia.
- En caso de llenarse el camión las empresas esperan a que haya espacio.
- Cada empresa tiene una capacidad de $venta máxima$.
- Cada comprador reliza un $compra total$.
- De no haber mercancía en el camión los compradores esperarán hasta que haya.

In [44]:
import multiprocessing as mp 
import time 
import random 
from sys import stdout 

class Empresa(mp.Process):
  def __init__(self, id, venta_maxima, camion, en_camion, mutex, espacios_vacios):
    mp.Process.__init__(self)
    self.id = id 
    self.venta_maxima = venta_maxima 
    self.venta_actual = 0
    self.camion = camion
    self.en_camion = en_camion
    self.mutex = mutex 
    self.espacios_vacios = espacios_vacios

  def vender_naranjas(self):
    self.espacios_vacios.acquire()
    self.mutex.acquire()
    self.camion.put(1)
    self.mutex.release()
    self.en_camion.release()

  def run(self):
    print(f"Empresa {self.id} ha comenzado a vender y tiene hasta {self.venta_maxima} cajas de naranjas")
    while self.venta_actual < self.venta_maxima:
      self.vender_naranjas()
      print(f"Empresa {self.id} acaba de vender una caja de naranjas")
      self.venta_maxima += 1
      time.sleep(1)

class Comprador(mp.Process):
  def __init__(self, id, compra_total, camion, en_camion, mutex, espacios_vacios):
    mp.Process.__init__(self)
    self.id = id
    self.compra_total = compra_total 
    self.compra_actual = 0
    self.camion = camion
    self.en_camion = en_camion
    self.mutex = mutex
    self.espacios_vacios = espacios_vacios

  def comprar_naranjas(self):
    en_camion.acquire()
    self.mutex.acquire()
    print(f"Comprador {id} acaba de comprar una caja de naranjas")
    caja_naranjas = self.camion.get()
    self.mutex.release()
    espacios_vacios.release()

  def run(self):
    print(f"Comprador {self.id} ha llegado y requiere {self.compra_total} cajas de naranjas")
    while self.compra_actual < self.compra_total:
      self.comprar_naranjas()
      self.compra_actual += 1
 
class Central():
  def __init__(self, capacidad_camion, n_empresas, n_compradores):
    self.n_empresas = n_empresas
    self.n_compradores = n_compradores 
    self.camion = mp.Queue()
    self.en_camion = mp.Semaphore(0)
    self.espacios_vacios = mp.Semaphore(capacidad_camion)
    self.mutex = mp.Lock()

  def comenzar_dia(self):
    self.empresas = [Empresa(i, random.randint(1, 5), self.camion, self.en_camion, self.mutex, self.espacios_vacios) for i in range(self.n_empresas)]
    self.compradores = [Comprador(i, random.randint(1, 2), self.camion, self.en_camion, self.mutex, self.espacios_vacios) for i in range(self.n_compradores)]
    for empresa in self.empresas:
      empresa.start()
    for comprador in self.compradores:
      comprador.start()

central_abastos = Central(capacidad_camion=10, n_empresas=2, n_compradores=3)
central_abastos.comenzar_dia()

Empresa 0 ha comenzado a vender y tiene hasta 5 cajas de naranjas
Empresa 0 acaba de vender una caja de naranjas
Empresa 1 ha comenzado a vender y tiene hasta 5 cajas de naranjas
Empresa 1 acaba de vender una caja de naranjas
Comprador 1 ha llegado y requiere 1 cajas de naranjas
Comprador 0 ha llegado y requiere 1 cajas de naranjas
Comprador 2 ha llegado y requiere 2 cajas de naranjas
